In [3]:
from pathlib import Path
import sys
path_root = Path().absolute().parent
sys.path.append(str(path_root))

import numpy as np
import pandas as pd
import pickle
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

from radiomicsutil.voi import find_contours, getMask, calc_vol, calc_mesh, smooth_mesh, plot_mesh, plot_mesh3d
from radiomicsutil.threed import ThreedMorphologyFeaturesExtractor

In [4]:
with open("./ISD-0003_slicelocations.pkl", "rb") as f:
    case_slice_locations = pickle.load(f)

with open("./ISD-0003_pixelspacing.pkl", "rb") as f:
    case_pixel_spacings = pickle.load(f)

In [ ]:
def caseExtractFeatures(isd_number, image_shape=(800,800)):
    print(isd_number)
    # case = Path(f"/home/azetry/datasets/TACE/{isd_number}_result")
    
    pixel_spacing_xy = case_pixel_spacings[isd_number][0]

    # Convert slice locations to float and sort along with masks
    slice_locations = case_slice_locations[isd_number]
    sorted_indices = np.argsort(slice_locations)
    slice_locations.sort()


    # with open("./ISD-0003/imgs.npy", "rb") as f:
    #     liver_preds = pickle.load(f)
    npy_file_path = './ISD-0003/imgs.npy'
    liver_preds = np.load(npy_file_path)

    tumor_preds = dict()
    for png in case.glob("./*.png"):
        idx = int(png.name.replace(".png",""))
        img = cv2.imread(str(png))
        tumor_preds[idx] = find_contours(img)

    # liver_masks = [ getMask(ctr,image_shape) for ctr in liver_preds ]
    # liver_masks = [liver_masks[i] for i in sorted_indices]

    tumor_masks = [ getMask(tumor_preds[idx],image_shape) for idx in range(len(tumor_preds)) ] # 因為排序不一定照排序
    tumor_masks = [tumor_masks[i] for i in sorted_indices]

    # Default rescale to 1 mm
    # liver_vol = calc_vol(liver_masks, slice_locations, pixel_spacing_xy)
    tumor_vol = calc_vol(tumor_masks, slice_locations, pixel_spacing_xy)

    # print("Building Liver Model...")
    # liver_extractor = ThreedMorphologyFeaturesExtractor(liver_vol)
    # print("Done.")
    # print("Building Tumor Model...")
    tumor_extractor = ThreedMorphologyFeaturesExtractor(tumor_vol)

    threed_morphology_p = tumor_extractor.all('p')
    
    # 用 triangle mesh 計算volume所花成本和時間太大，直接用 voxels 計算近似值 (因為我們已經rescale 到 1 mm 所以不用擔心)
    # liver_voxels = np.sum(liver_vol)
    tumor_voxels = np.sum(tumor_extractor.voi_voxels)
    # print(liver_voxels, tumor_voxels)
    print(tumor_voxels)
    # print(f"Liver voxels:{liver_voxels}")
    # threed_morphology_p['ｐ_tumor_liver_volume_ratio'] = pd.Series([tumor_voxels / liver_voxels])
    # threed_morphology_p['p_tumor_liver_diff'] = pd.Series([liver_voxels-tumor_voxels])
    # threed_morphology_p['p_tumor_liver_diff_ratio'] = threed_morphology_p['p_tumor_liver_diff'] / liver_voxels

    threed_morphology_p = threed_morphology_p.set_index(pd.Index([f'{isd_number}']))
    return threed_morphology_p
    